In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter


from quick_sim import setup_sim
# import edward_tools.fq_runner as fq_runner
from edward_tools.simple_1D_potential import flux_qubit_1D_non_linear_approx_pot, flux_qubit_1D_non_linear_approx_force
import kyle_tools as kt
import matplotlib.pyplot as plt

In [ ]:
has_velocity = True # True: underdamped dynamics, False: overdamped dyanmics

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 4.2
# T = 7
k_BT = k_B * T
C_factor, R_factor, L_factor = 1, 1, 1

R_1 = 100   * R_factor # ohm
C_1 = 1e-12 * C_factor # F
L_1 = 5e-12 * L_factor # Inductor L1, L_2 H 
I_m_1 = 0
freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)
print(f"freq = {freq / 1e9:.3f}GHz, characteristic_time  = {characteristic_time * 1e12:.3f}ps")


In [ ]:
m_c = C_1
m_1 = C_1
x_c = PHI_0 / (2 * np.pi) # position constant
t_c = np.sqrt(L_1 * C_1) # time constant
v_c = x_c / t_c # speed constant

U0_1 = m_c * x_c**2 / t_c**2 / k_BT   # energy scale
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

gamma = 16
beta_1 = 2.3
d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 


_lambda = np.array([lambda_1])
_theta  = np.array([theta_1])
_eta  =   np.array([eta_1])

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 4000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1]
params['mass'] = np.array([1])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::500] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
    
initial_parameter_dict = {
        "U0_1": U0_1,     "gamma": gamma,  "beta": beta_1,   "d_beta": d_beta_1,
        "phi_1x": 0,  "phi_1xdc": 0, 'x_c': x_c, 'shift': 0
}

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [U0_1, gamma, beta_1, d_beta_1, 0, 0, x_c, 0]
[phi_1_bound, phi_1dc_bound] = np.array([4, 4])
contour_range = [300, 2000]

coupled_fq_domain = [[-phi_1_bound], [phi_1_bound]]

coupled_fq_pot = Potential(flux_qubit_1D_non_linear_approx_pot, flux_qubit_1D_non_linear_approx_force, 8, 1, default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

In [ ]:
# to turn on/off dissipation and noise
zeroDissipation = False
# zeroDissipation = True
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['sim_params'] = [_lambda, _theta, _eta]

# save the states at all time steps
saveAllStates = True

In [ ]:
def create_one_bit_Protocol(_t, _protocol):
    _protocol["duration"] = _t
    return _protocol

protocol_list = [   
    create_one_bit_Protocol(np.pi, {
    "phi_1x": 0, "phi_2x": 0, "mu_12": 0, \
    "phi_1xdc": 0, "phi_2xdc": 0, "shift": 1, "name":"decrease k"
    }),
]

# setting initial states

# create cfqr object

In [ ]:
protocol_key = ['U0_1', 'gamma', 'beta', 'd_beta', 'phi_1x', 'phi_1xdc', 'x_c', 'shift']


def customizedProtocol(initial_values_dict, protocol_list):
    """
    Constructs a dictionary of protocol parameters over time based on a list of durations and target values.

    Args:
        initial_values_dict (dict): Initial values for each parameter (e.g., {'phi_1': 0.0, 'phi_1x': 1.0}).
        protocol_list (list): A list of dictionaries, each containing 'duration' and any parameters to update.
        normalized (bool): If True, scales the time axis 't' from 0 to 1.

    Returns:
        dict: A dictionary where keys are parameter names (plus 't') and values are lists of settings at each step.
    """    
    protocol_parameter_dict = {key: [value] for key, value in initial_values_dict.items()}
    # Start the time tracking at 0.0
    protocol_parameter_dict["t"] = [0.0]

    # Iterate through each step in the provided protocol list
    for item in protocol_list:
        protocol_parameter_dict["t"].append(protocol_parameter_dict["t"][-1] + item["duration"])

        # Update all parameters defined in the initial set for this duration
        for key in initial_values_dict.keys():
            if key in item:
                protocol_parameter_dict[key].append(item[key])
            else:
                protocol_parameter_dict[key].append(protocol_parameter_dict[key][-1])

    return protocol_parameter_dict


def create_system(protocol_parameter_dict, domain = None, modifiedFunction = None):
    """
    This function is used to produce the storage and computation protocol

    input:
    1. protocol_parameter_dict:
    - a dictionary contains the an array of time, which represents the time point at which the protocol is changed
    - the key is the name of the parameter
    - for parameters, they are arrays containing the value of the parameter at the particular time point

    2. modifiedFunction:
    - To modify the simple linear parametrization of the value
    - You can use more complex method to get the values such as calculate the value of phi_1x based on the value of other parameters
    
    output:
    1. comp_protocol: the protocol for the computation system
    2. storage_protocol: : the protocol for the equilibrium system
    """
    # --- STORAGE PROTOCOL CONSTRUCTION ---
    # The storage protocol represents a simple transition from the very beginning to the very end.
    # It ignores all intermediate steps in the protocol_parameter_dict.
    storage_t = (protocol_parameter_dict["t"][0], protocol_parameter_dict["t"][-1])
    storage_protocol_parameter_time_series = [np.array([protocol_parameter_dict[key][0], protocol_parameter_dict[key][-1]]) for key in protocol_key]
    storage_protocol_parameter_time_series = np.array(storage_protocol_parameter_time_series)
    storage_protocol = Protocol(storage_t, storage_protocol_parameter_time_series)
    storage_protocol.modifiedFunction = None

    # --- COMPUTATION PROTOCOL CONSTRUCTION ---
    # The computation protocol is more detailed, consisting of multiple segments 
    # that follow every step defined in the protocol_parameter_dict.
    comp_protocol_array = []
    comp_t = protocol_parameter_dict["t"]
    comp_protocol_parameter_time_series = [protocol_parameter_dict[key] for key in protocol_key]
    comp_protocol_parameter_time_series = np.array(comp_protocol_parameter_time_series).T

    if modifiedFunction == None:
        modifiedFunction = [None for _ in range(len(comp_t)-1)]
    
    # Loop through each time interval to create individual Protocol segments
    for i in range(len(comp_t)-1):
        n_th_comp_time_array = (comp_t[i], comp_t[i+1])
        n_th_comp_protocol_parameter_array = np.array([comp_protocol_parameter_time_series[i], comp_protocol_parameter_time_series[i+1]]).T # in the form of array of [(p_n_i, p_n_f)]
        _p = Protocol(n_th_comp_time_array, n_th_comp_protocol_parameter_array)
        
        _p.modifiedFunction = modifiedFunction[i]
        comp_protocol_array.append(_p)
    comp_protocol = Compound_Protocol(comp_protocol_array)
    comp_protocol.protocol_array  = comp_protocol_array

    return storage_protocol, comp_protocol

In [ ]:
from edward_tools.fq_runner import fluxQubitRunner

computation_protocol_parameter_dict = customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)






In [ ]:
fq_runner = fluxQubitRunner(potential=coupled_fq_pot, params=params, potential_default_param_dict=initial_parameter_dict, storage_protocol=storage_protocol, computation_protocol=comp_protocol)
fq_runner.initialize_sim()
# fq_runner.set_sim_attributes()

In [ ]:
simResult = fq_runner.run_sim()

In [ ]:
phi_1_array = np.linspace(-4, 4, 100)
all_states = fq_runner.sim.output.all_state["states"]

plt.hist(all_states[:, 0, 0, 0], bins = 100)
bins = np.linspace(-4, 4, 200)
phi_1_array = np.linspace(-4, 4, 100)

fig, ax = plt.subplots()
pdf_ax = ax.twinx()
p_0_index = all_states[:, 0, 0, 0] < 0
p_1_index = all_states[:, 0, 0, 0] > 0
p_0_color, p_1_color = "red", "blue"


# plot the potential and distribution historgram at t = 0
params_at_0 = fq_runner.protocol.get_params(0)
potential_at_t_0 = flux_qubit_1D_non_linear_approx_pot(phi_1_array, params_at_0)


ax.plot(phi_1_array, potential_at_t_0 -  min(potential_at_t_0), color='red')
plt.hist(fq_runner.sim.output.all_state["states"][p_0_index, 0, 0, 0], bins = bins, color = p_0_color)
plt.hist(fq_runner.sim.output.all_state["states"][p_1_index, 0, 0, 0], bins = bins, color = p_1_color)
pdf_ax.get_yaxis().set_visible(False)
pdf_ax.set_xlim(-4, 4)


# plot the potential and distribution historgram at time step t
def update(i):
    ax.clear()
    pdf_ax.clear()
    ax.set_ylabel('Potential (k_BT)')

    params_at_t_i = fq_runner.protocol.get_params(i * params["dt"])
    potential_at_t_i = flux_qubit_1D_non_linear_approx_pot(phi_1_array, params_at_t_i)
    potential_at_t_i = potential_at_t_i - min(potential_at_t_0)
    ax.plot(phi_1_array, potential_at_t_i, color='red')

    pdf_ax.hist(all_states[p_0_index, i, 0, 0], bins = bins, color = p_0_color)
    pdf_ax.hist(all_states[p_1_index, i, 0, 0], bins = bins, color = p_1_color)
    pdf_ax.set_xlim(-4, 4)
    ax.set_title(f"Kyle's simulator (t = {params['dt'] * i:.3g})")


animation_frames = range(0, all_states.shape[1], 5)
ani = animation.FuncAnimation(fig, update, frames=animation_frames, interval = 50)

plt.close()
# HTML(ani.to_jshtml())
HTML(ani.to_html5_video())

In [ ]:
# plot the work cost
W_3_std = fq_runner.sim.work_statistic_array[-1][1]/np.sqrt(params["N"]) * 3
plt.hist(fq_runner.sim.work_dist_array, bins = 2)
plt.title(f"mean work = {np.mean(fq_runner.sim.work_dist_array):.3g} ± {W_3_std:.3g} k_BT") 
plt.xlabel("W (k_BT)")
plt.ylabel("count")